In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from bikewheelcalc import BicycleWheel, Rim, Hub, ModeMatrix

%matplotlib inline

In [ ]:
radius = 0.305
mass = 0.540


w = BicycleWheel()
w.hub = Hub(diameter=0.050, width=0.050)
w.rim = Rim(radius=radius, area=mass / (2*np.pi*radius*2700.),
            I_rad=110./69e9, I_lat=220./69e9, J_tor=25./26e9, I_warp=0.0,
            young_mod=69e9, shear_mod=26e9)

w.lace_cross(n_spokes=36, n_cross=3, diameter=2.0e-3, young_mod=210e9, offset=0.);

In [ ]:
def calc_dT(mm, d):
    return np.array([-s.EA/s.length/9.81 *
                     np.dot(s.n, mm.B_theta(s.rim_pt[1], comps=[0, 1, 2]).dot(dm))
                     for s in mm.wheel.spokes])

def plot_dT(dT, ax):
    ax.bar([s.rim_pt[1] - np.pi/2 for s in w.spokes[::2]],
            dT[::2], width=0.08, bottom=100.0, color='C0')
    ax.bar([s.rim_pt[1] - np.pi/2 for s in w.spokes[1::2]],
            dT[1::2], width=0.08, bottom=100.0, color='C1')


mm = ModeMatrix(w, N=36)

K = (mm.K_rim(tension=True, r0=True) +
     mm.K_spk(tension=True, smeared_spokes=False))

fig = plt.figure(figsize=(8, 3))


# Radial force
F_ext = mm.F_ext(f_theta=0., f=[0., 60*9.81, 0., 0.])
dm = np.linalg.solve(K, F_ext)
dT = calc_dT(mm, dm)
print(np.max(dT))

ax1 = plt.subplot(131, projection='polar')
plot_dT(dT, ax1)


# Lateral force
F_ext = mm.F_ext(f_theta=0., f=[20*9.81, 0., 0., 0.])
dm = np.linalg.solve(K, F_ext)
dT = calc_dT(mm, dm)
print(np.max(dT))

ax2 = plt.subplot(132, projection='polar')
plot_dT(dT, ax2)


# Tangential force
F_ext = mm.F_ext(f_theta=0., f=[0., 0., 20*9.81, 0.])
dm = np.linalg.solve(K, F_ext)
dT = calc_dT(mm, dm)
print(np.max(dT))

ax3 = plt.subplot(133, projection='polar')
plot_dT(dT, ax3)


for a in [ax1, ax2, ax3]:
    
    a.plot(np.linspace(0., 2*np.pi, 200), 100.*np.ones(200), 'k', linewidth=0.5)
    a.set_ylim([0., 135.])
    a.set_xticks([])
    a.set_yticks([])
    a.spines['polar'].set_visible(False)
    
plt.tight_layout()